# Day 23
## Part 1

Please don't read this code, it's a mess.

Try Dijkstra's algorithm with amphipod movement rules and a couple of optimisations. Use pyrsistent where it's needed, i.e. in the set of seen states, but only then as using it universally slows things down. 

In [1]:
from collections import defaultdict, namedtuple
import pyrsistent as pyr
from heapq import heapify, heappop, heappush

Edge = namedtuple('Edge', 'neighbour, length')

def create_graph():
    g = defaultdict(list)
    top_row = [1,2,4,6,8,10,11]
    for i in range(len(top_row) - 1):
        g[str(top_row[i])].append(Edge(str(top_row[i + 1]), top_row[i + 1] - top_row[i]))
        g[str(top_row[i + 1])].append(Edge(str(top_row[i]), top_row[i + 1] - top_row[i]))
    for xs, y in [
        (("2", "4"), "A1"),
        (("4", "6"), "B1"),
        (("6", "8"), "C1"),
        (("8", "10"), "D1")
    ]:
        for x in xs:
            g[x].append(Edge(y, 2))
            g[y].append(Edge(x, 2))
    for c in "ABCD":
        g[f"{c}1"].append(Edge(f"{c}2", 1))
        g[f"{c}2"].append(Edge(f"{c}1", 1))
    return g

def parse_data(s):
    lines = s.strip().splitlines()
    room_locations = {
        "A1": (2, 3),
        "A2": (3, 3),
        "B1": (2, 5),
        "B2": (3, 5),
        "C1": (2, 7),
        "C2": (3, 7),
        "D1": (2, 9),
        "D2": (3, 9)
    }
    burrow = {}
    for room in room_locations:
        row, col = room_locations[room]
        burrow[room] = lines[row][col]
    return burrow

test_data = parse_data('''
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
''')

test_data

{'A1': 'B',
 'A2': 'A',
 'B1': 'C',
 'B2': 'D',
 'C1': 'B',
 'C2': 'C',
 'D1': 'D',
 'D2': 'A'}

In [2]:
from functools import total_ordering
from dataclasses import dataclass

@total_ordering
@dataclass
class Burrow:
    energy: int
    positions: dict
        
    def __eq__(self, other):
        return self.energy == other.energy
    
    def __lt__(self, other):
        return self.energy < other.energy 
    
    def __iter__(self):
        return iter((self.energy, self.positions))
    

def possible_path(node, graph, positions):
    # All possible paths, whether or not they are legitimate
    paths = []
    seen = set()
    to_see = [(0, node)]
    amphipod = positions[node]
    energy_per_step = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}[amphipod] 
    while to_see:
        energy_so_far, this_node = heappop(to_see)
        if this_node not in seen:
            seen.add(this_node)
            for edge in graph[this_node]:
                if edge.neighbour not in positions and edge.neighbour not in seen:
                    paths.append((edge.neighbour, energy_so_far + edge.length * energy_per_step))
                    heappush(to_see, (energy_so_far + edge.length * energy_per_step, edge.neighbour))
              
    # Which paths are legitimate?
    for next_node, energy in paths:
        node_is_hallway = node.isdigit()
        next_node_is_hallway = next_node.isdigit()
        amphipods_room = next_node[0] == amphipod
        amphipods_room_clear = all(
            positions.get(f'{amphipod}{c}', amphipod) == amphipod
            for c in '12'
        )
        already_in_final_room = node[0] == amphipod
        end_of_amphipods_room = next_node == max([f'{amphipod}{c}' for c in '12'
                                                  if f'{amphipod}{c}' not in positions],
                                                 default = None)
        legit = False
        if not node_is_hallway and next_node_is_hallway:
            legit = True
        if amphipods_room and amphipods_room_clear and end_of_amphipods_room:
            legit = True
        if already_in_final_room and amphipods_room_clear and next_node_is_hallway:
            legit = False
        if legit:
            yield next_node, energy
            

def finished(positions):
    return all(
        p[0] == positions[p] for p in positions
    )

def part_1(graph, positions):
    queue = [(0, positions)]
    seen = set()
    while queue:
        energy_so_far, pos_so_far = heappop(queue)
        if pyr.pmap(pos_so_far) not in seen:
            seen.add(pyr.pmap(pos_so_far))
            if finished(pos_so_far):
                return energy_so_far
            for node in pos_so_far:
                amphipod = pos_so_far[node]
                for next_node, next_energy in possible_path(node, graph, pos_so_far):
                    next_pos = pos_so_far.copy()
                    del next_pos[node]
                    next_pos[next_node] = amphipod
                    if pyr.pmap(next_pos) not in seen:
                        heappush(
                            queue,
                            Burrow(energy_so_far + next_energy,
                                   next_pos)
                        )

graph = create_graph()

In [3]:
part_1(graph, test_data)

12521

In [4]:
%%timeit 
part_1(graph, test_data)

14.5 s ± 365 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
data = parse_data(open('input', 'r').read())
part_1(graph, data)

18051

## Part 2
That's already a bit slow so use an A* algorithm with a heuristic of the minimum energy needed for each amphipod not in a clean room to reach the clean room.

First generalise the graph.

In [39]:
def create_graph(room_size=2):
    g = defaultdict(list)

    top_row = [1,2,4,6,8,10,11]
    for i in range(len(top_row) - 1):
        g[str(top_row[i])].append(Edge(str(top_row[i + 1]), top_row[i + 1] - top_row[i]))
        g[str(top_row[i + 1])].append(Edge(str(top_row[i]), top_row[i + 1] - top_row[i]))

    for xs, y in [
        (("2", "4"), "A1"),
        (("4", "6"), "B1"),
        (("6", "8"), "C1"),
        (("8", "10"), "D1")
    ]:
        for x in xs:
            g[x].append(Edge(y, 2))
            g[y].append(Edge(x, 2))

    for i in range(1, room_size):
        for c in "ABCD":
            g[f"{c}{i}"].append(Edge(f"{c}{i + 1}", 1))
            g[f"{c}{i + 1}"].append(Edge(f"{c}{i}", 1))
            
    return g


def parse_data_2(s):
    lines = s.strip().splitlines()
    lines = lines[:3] + ["  #D#C#B#A#", "  #D#B#A#C#"] + lines[3:]
    room_length = len(lines) - 3
    room_locations = {}

    for letter, col in zip('ABCD', range(3, 10, 2)):
        for i in range(1, room_length + 1):
            row = 1 + i
            room_locations[f'{letter}{i}'] = (row, col)

    burrow = {}
    for room in room_locations:
        row, col = room_locations[room]
        burrow[room] = lines[row][col]
    return burrow


test_data_2 = parse_data_2('''
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
''')

test_data_2

{'A1': 'B',
 'A2': 'D',
 'A3': 'D',
 'A4': 'A',
 'B1': 'C',
 'B2': 'C',
 'B3': 'B',
 'B4': 'D',
 'C1': 'B',
 'C2': 'B',
 'C3': 'A',
 'C4': 'C',
 'D1': 'D',
 'D2': 'A',
 'D3': 'C',
 'D4': 'A'}

Generalise the `possible_path` function.

In [7]:
def possible_path(node, graph, positions):
    # All possible paths, whether or not they are legitimate
    paths = []
    seen = set()
    to_see = [(0, node)]
    amphipod = positions[node]
    energy_per_step = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}[amphipod] 
    while to_see:
        energy_so_far, this_node = heappop(to_see)
        if this_node not in seen:
            seen.add(this_node)
            for edge in graph[this_node]:
                if edge.neighbour not in positions and edge.neighbour not in seen:
                    paths.append((edge.neighbour, energy_so_far + edge.length * energy_per_step))
                    heappush(to_see, (energy_so_far + edge.length * energy_per_step, edge.neighbour))

    # Which paths are legitimate?
    room_numbers = ''.join(n[1] for n in graph if n[0] == 'A')
    for next_node, energy in paths:
        node_is_hallway = node.isdigit()
        next_node_is_hallway = next_node.isdigit()
        amphipods_room = next_node[0] == amphipod
        amphipods_room_clear = all(
            positions.get(f'{amphipod}{c}', amphipod) == amphipod
            for c in room_numbers
        )
        already_in_final_room = node[0] == amphipod
        end_of_amphipods_room = next_node == max([f'{amphipod}{c}' for c in room_numbers
                                                  if f'{amphipod}{c}' not in positions],
                                                 default = None)
        legit = False
        if not node_is_hallway and next_node_is_hallway:
            legit = True
        if amphipods_room and amphipods_room_clear and end_of_amphipods_room:
            legit = True
        if already_in_final_room and amphipods_room_clear and next_node_is_hallway:
            legit = False
        if legit:
            yield next_node, energy

Test they're still working.

In [8]:
graph = create_graph()
part_1(graph, test_data)

12521

For the heuristic function we'll need a matrix of the distance between points, which is kept within a closure.

In [54]:
from collections import deque

def heuristic(graph):
    m = {}
    for n in graph:
        seen = set()
        queue = deque([(n, 0)])
        while queue:
            this_node, length = queue.popleft()
            if this_node not in seen:
                seen.add(this_node)
                m[n, this_node] = length
                for next_node, l in graph[this_node]:
                    queue.append((next_node, length + l))
    energy_per_step = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

    def h(positions):
        min_length = 0
        for p in positions:
            amphipod = positions[p]
            if amphipod != p[0]:
                min_length += m[(p, f'{amphipod}{1}')] * energy_per_step[amphipod]
        # Count the additional steps if more than one amphipod of each type 
        # is outside the room
        for a in 'ABCD':
            n = len([positions[p] 
                     for p in positions 
                     if positions[p] == a and p[0] != positions[p]])
            min_length += energy_per_step[a] * n * (n - 1) // 2
        return min_length
    
    return h


def min_energy(graph, positions):
    h = heuristic(graph)
    queue = [(h(positions), (0, positions))]
    seen = set()
    while queue:
        _, (energy_so_far, pos_so_far) = heappop(queue)
        if pyr.pmap(pos_so_far) not in seen:
            seen.add(pyr.pmap(pos_so_far))
            if finished(pos_so_far):
                return energy_so_far
            for node in pos_so_far:
                amphipod = pos_so_far[node]
                for next_node, next_energy in possible_path(node, graph, pos_so_far):
                    next_pos = pos_so_far.copy()
                    del next_pos[node]
                    next_pos[next_node] = amphipod
                    if pyr.pmap(next_pos) not in seen:
                        heappush(
                            queue,
                            (
                                energy_so_far + next_energy + h(next_pos),
                                Burrow(energy_so_far + next_energy,
                                       next_pos)
                            )
                        )

                        
def part_1_opt(positions):
    return min_energy(create_graph(), positions)

In [55]:
part_1_opt(test_data)

12521

Correct, and quite a bit quicker.

In [56]:
%%timeit 
part_1_opt(test_data)

1.07 s ± 30.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
def part_2(positions):
    return min_energy(create_graph(4), positions)

part_2(test_data_2)

44169

Correct on the test data, though quite slow.

In [63]:
%%timeit
part_2(test_data_2)

44 s ± 1.06 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
data_2 = parse_data_2(open('input', 'r').read())

In [59]:
part_2(data)

22495

But incorrect on the actual data for at least the third time this year. Nnngh. 